# FedCAD: Centralized vs Federated (orchestration)

This notebook launches centralized and federated experiments via existing entry points:
- Centralized: `python train_central.py`
- Federated: `flwr run .`

Metrics tracked in W&B:
- Centralized: `train_loss`, `test_loss`, `test_acc` (already logged by `train_central.py`)
- Federated: depends on your Flower app (ensure it logs to the same W&B project/group)

Recommended seeds for a seminar: 3 (e.g., `[0, 1, 2]`). Use 5 if time permits for tighter confidence intervals.

In [4]:
import os, sys, subprocess, shutil, time
import torch

# W&B configuration
def load_wandb_config():
    """Load WandB credentials from config file (optional for reproducibility)"""
    config = {}
    config_path = './.wandb_config'
    try:
        with open(config_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    config[key] = value
        return config.get('WANDB_ENTITY'), config.get('WANDB_PROJECT')
    except FileNotFoundError:
        return None, None

entity, project_name = load_wandb_config()
WANDB_MODE = "online"               # set to "offline" or "disabled" if needed

# Experiment seeds (3 is seminar-friendly)
SEEDS = [0, 1, 2]

# Centralized hyperparameters
EPOCHS_CENTRAL = 10
LR_CENTRAL = 1e-3
EVAL_INTERVAL = 1

# Device
USE_GPU_FLAG = torch.cuda.is_available()
print("Device:", "cuda" if USE_GPU_FLAG else "cpu")
print("Seeds:", SEEDS)

def run_cmd(cmd_list, extra_env=None, cwd=None):
    env = os.environ.copy()
    if extra_env:
        env.update({k: str(v) for k, v in extra_env.items()})
    print("\n>>>", " ".join(cmd_list))
    print("    in:", cwd or os.getcwd())
    print("    with env overrides:", extra_env or {})
    proc = subprocess.Popen(cmd_list, cwd=cwd, env=env)
    proc.wait()
    if proc.returncode != 0:
        raise RuntimeError(f"Command failed with exit code {proc.returncode}: {' '.join(cmd_list)}")

Device: cpu
Seeds: [0, 1, 2]


## Centralized experiments

Runs `train_central.py` once per seed. The script already logs: `train_loss`, `test_loss`, `test_acc`.

Note: `train_central.py` does not expose a `--seed` flag. If you need strict reproducibility per seed, add seeding to the script; otherwise each run will use random initialization.

In [5]:
for seed in SEEDS:
    # W&B grouping so runs are easy to compare
    env_overrides = {
        "WANDB_PROJECT": project_name,
        "WANDB_MODE": WANDB_MODE,
        "WANDB_RUN_GROUP": "centralized",
        "WANDB_TAGS": "centralized,notebook",
        "WANDB_ENTITY": entity,

        # Optional: many apps read SEED from env; train_central.py currently does not
        "SEED": seed,
    }

    cmd = [sys.executable, "train_central.py",
           "--epochs", str(EPOCHS_CENTRAL),
           "--lr", str(LR_CENTRAL),
           "--eval-interval", str(EVAL_INTERVAL),
           "--wandb-mode", WANDB_MODE]
    if USE_GPU_FLAG:
        cmd.append("--gpu")

    print(f"\n=== Centralized run, seed={seed} ===")
    run_cmd(cmd, extra_env=env_overrides, cwd=os.getcwd())
    time.sleep(1)  # small gap between runs


=== Centralized run, seed=0 ===

>>> /Users/zoe/Desktop/Fed-CAD/.venv/bin/python train_central.py --epochs 10 --lr 0.001 --eval-interval 1 --wandb-mode online
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'centralized', 'WANDB_TAGS': 'centralized,notebook', 'WANDB_ENTITY': 'louisewiljander-ludwig-maximilianuniversity-of-munich', 'SEED': 0}


wandb: Currently logged in as: zoematr (louisewiljander-ludwig-maximilianuniversity-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: setting up run l8i2ow7z
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /Users/zoe/Desktop/Fed-CAD/wandb/run-20260121_150942-l8i2ow7z
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run centralized
wandb: ⭐️ View project at https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD
wandb: 🚀 View run at https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD/runs/l8i2ow7z


Epoch 1/10 | train_loss=0.6267 | test_loss=0.4964 | test_acc=0.7636
Epoch 2/10 | train_loss=0.5823 | test_loss=0.4664 | test_acc=0.7636
Epoch 3/10 | train_loss=0.5408 | test_loss=0.4809 | test_acc=0.7636
Epoch 4/10 | train_loss=0.5367 | test_loss=0.5275 | test_acc=0.7636
Epoch 5/10 | train_loss=0.5392 | test_loss=0.4476 | test_acc=0.7727
Epoch 6/10 | train_loss=0.5158 | test_loss=0.4406 | test_acc=0.7818
Epoch 7/10 | train_loss=0.4898 | test_loss=0.4622 | test_acc=0.7909
Epoch 8/10 | train_loss=0.4789 | test_loss=0.4214 | test_acc=0.8000
Epoch 9/10 | train_loss=0.4696 | test_loss=0.4277 | test_acc=0.7909
Epoch 10/10 | train_loss=0.4746 | test_loss=0.4905 | test_acc=0.8000
Saved model -> models/final_model_centralized.pt


wandb: updating run metadata
wandb: uploading output.log; uploading wandb-summary.json; uploading config.yaml
wandb: uploading history steps 1-9, summary, console lines 0-10
wandb: 
wandb: Run history:
wandb:      epoch ▁▂▃▃▄▅▆▆▇█
wandb:   test_acc ▁▁▁▁▃▅▆█▆█
wandb:  test_loss ▆▄▅█▃▂▄▁▁▆
wandb: train_loss █▆▄▄▄▃▂▁▁▁
wandb: 
wandb: Run summary:
wandb:            epoch 10
wandb:   final_test_acc 0.8
wandb:  final_test_loss 0.4905
wandb: final_train_loss 0.47461
wandb:         test_acc 0.8
wandb:        test_loss 0.4905
wandb:       train_loss 0.47461
wandb: 
wandb: 🚀 View run centralized at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD/runs/l8i2ow7z
wandb: ⭐️ View project at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20260121_150942-l8i2ow7z/logs



=== Centralized run, seed=1 ===

>>> /Users/zoe/Desktop/Fed-CAD/.venv/bin/python train_central.py --epochs 10 --lr 0.001 --eval-interval 1 --wandb-mode online
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'centralized', 'WANDB_TAGS': 'centralized,notebook', 'WANDB_ENTITY': 'louisewiljander-ludwig-maximilianuniversity-of-munich', 'SEED': 1}


wandb: Currently logged in as: zoematr (louisewiljander-ludwig-maximilianuniversity-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: setting up run piyg2tn4
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /Users/zoe/Desktop/Fed-CAD/wandb/run-20260121_150953-piyg2tn4
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run centralized
wandb: ⭐️ View project at https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD
wandb: 🚀 View run at https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD/runs/piyg2tn4


Epoch 1/10 | train_loss=0.6307 | test_loss=0.5052 | test_acc=0.7636
Epoch 2/10 | train_loss=0.5955 | test_loss=0.5778 | test_acc=0.7636
Epoch 3/10 | train_loss=0.5859 | test_loss=0.5052 | test_acc=0.7636
Epoch 4/10 | train_loss=0.5694 | test_loss=0.4673 | test_acc=0.7636
Epoch 5/10 | train_loss=0.5457 | test_loss=0.4445 | test_acc=0.7636


Traceback (most recent call last):
  File "/Users/zoe/Desktop/Fed-CAD/train_central.py", line 97, in <module>
    train_central(args)
  File "/Users/zoe/Desktop/Fed-CAD/train_central.py", line 46, in train_central
    for images, labels in trainloader:
                          ^^^^^^^^^^^
  File "/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 733, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 789, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 50, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoe/Desktop/Fed-CAD/

KeyboardInterrupt: 

## Federated experiments (Flower Simulation Engine)

Runs `flwr run .` once per seed. Ensure your Flower app logs to W&B (same project) with metrics comparable to centralized (e.g., `global_train_loss`, `global_test_loss`, `global_test_acc`).

Tip: If your Flower app reads `WANDB_*` env vars and `SEED`, the grouping and seeds below will apply.

In [6]:
def run_flower(extra_env=None):
    # Prefer CLI if available; fallback to `python -m flwr`
    if shutil.which("flwr"):
        run_cmd(["flwr", "run", "."], extra_env=extra_env, cwd=os.getcwd())
    else:
        run_cmd([sys.executable, "-m", "flwr", "run", "."], extra_env=extra_env, cwd=os.getcwd())

for seed in SEEDS:
    env_overrides = {
        "WANDB_PROJECT": WANDB_PROJECT,
        "WANDB_MODE": WANDB_MODE,
        "WANDB_RUN_GROUP": "federated",
        "WANDB_TAGS": "federated,notebook",
        # Many FL apps use this to control randomness; ensure your app reads it
        "SEED": seed,
    }
    print(f"\n=== Federated run (Flower), seed={seed} ===")
    run_flower(extra_env=env_overrides)
    time.sleep(1)


=== Federated run (Flower), seed=0 ===

>>> flwr run .
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'federated', 'WANDB_TAGS': 'federated,notebook', 'SEED': 0}
Loading project configuration... 
Success


INFO :      Starting FedAvg strategy:
INFO :      	├── Number of rounds: 3
INFO :      	├── ArrayRecord (0.40 MB)
INFO :      	├── ConfigRecord (train): {'lr': 0.01}
INFO :      	├── ConfigRecord (evaluate): (empty!)
INFO :      	├──> Sampling:
INFO :      	│	├──Fraction: train (0.50) | evaluate ( 1.00)
INFO :      	│	├──Minimum nodes: train (2) | evaluate (2)
INFO :      	│	└──Minimum available nodes: 2
INFO :      	└──> Keys in records:
INFO :      		├── Weighted by: 'num-examples'
INFO :      		├── ArrayRecord key: 'arrays'
INFO :      		└── ConfigRecord key: 'config'
INFO :      
INFO :      
INFO :      [ROUND 1/3]
INFO :      configure_train: Sampled 5 nodes (out of 10)
INFO :      aggregate_train: Received 5 results and 0 failures
INFO :      	└──> Aggregated MetricRecord: {'train_loss': 0.8634607582951007}
INFO :      configure_evaluate: Sampled 10 nodes (out of 10)
INFO :      aggregate_evaluate: Received 10 results and 0 failures
INFO :      	└──> Aggregated MetricRecord: {'e


Saving final model to disk...

=== Federated run (Flower), seed=1 ===

>>> flwr run .
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'federated', 'WANDB_TAGS': 'federated,notebook', 'SEED': 1}
Loading project configuration... 
Success


INFO :      Starting FedAvg strategy:
INFO :      	├── Number of rounds: 3
INFO :      	├── ArrayRecord (0.40 MB)
INFO :      	├── ConfigRecord (train): {'lr': 0.01}
INFO :      	├── ConfigRecord (evaluate): (empty!)
INFO :      	├──> Sampling:
INFO :      	│	├──Fraction: train (0.50) | evaluate ( 1.00)
INFO :      	│	├──Minimum nodes: train (2) | evaluate (2)
INFO :      	│	└──Minimum available nodes: 2
INFO :      	└──> Keys in records:
INFO :      		├── Weighted by: 'num-examples'
INFO :      		├── ArrayRecord key: 'arrays'
INFO :      		└── ConfigRecord key: 'config'
INFO :      
INFO :      
INFO :      [ROUND 1/3]
INFO :      configure_train: Sampled 5 nodes (out of 10)
INFO :      aggregate_train: Received 5 results and 0 failures
INFO :      	└──> Aggregated MetricRecord: {'train_loss': 0.8279989625030959}
INFO :      configure_evaluate: Sampled 10 nodes (out of 10)
INFO :      aggregate_evaluate: Received 10 results and 0 failures
INFO :      	└──> Aggregated MetricRecord: {'e


Saving final model to disk...

=== Federated run (Flower), seed=2 ===

>>> flwr run .
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'federated', 'WANDB_TAGS': 'federated,notebook', 'SEED': 2}
Loading project configuration... 
Success


INFO :      Starting FedAvg strategy:
INFO :      	├── Number of rounds: 3
INFO :      	├── ArrayRecord (0.40 MB)
INFO :      	├── ConfigRecord (train): {'lr': 0.01}
INFO :      	├── ConfigRecord (evaluate): (empty!)
INFO :      	├──> Sampling:
INFO :      	│	├──Fraction: train (0.50) | evaluate ( 1.00)
INFO :      	│	├──Minimum nodes: train (2) | evaluate (2)
INFO :      	│	└──Minimum available nodes: 2
INFO :      	└──> Keys in records:
INFO :      		├── Weighted by: 'num-examples'
INFO :      		├── ArrayRecord key: 'arrays'
INFO :      		└── ConfigRecord key: 'config'
INFO :      
INFO :      
INFO :      [ROUND 1/3]
INFO :      configure_train: Sampled 5 nodes (out of 10)
INFO :      aggregate_train: Received 5 results and 0 failures
INFO :      	└──> Aggregated MetricRecord: {'train_loss': 1.127240760717541}
INFO :      configure_evaluate: Sampled 10 nodes (out of 10)
INFO :      aggregate_evaluate: Received 10 results and 0 failures
INFO :      	└──> Aggregated MetricRecord: {'ev


Saving final model to disk...
